# Real-world example: TelCo Call Center

This large notebook demonstrates how to apply this framework for build a real-world example: a call center for a telecommunications company. The core idea is to support users in their requests, such as:

- **Technical support**: users can call to ask for help with their internet connection, devices, or mobile.
- **Sales**: users can call to ask for new services, plans, or promotions.
- **Legal**: users can call to ask for information about their contracts, bills, or legal rights.

**NOTE**: an Azure AI Search instance is required to run the "Legal" agent, which performs RAG. A basic setup via ["Import and vectorize your data"](https://learn.microsoft.com/en-us/azure/search/search-get-started-portal-import-vectors?tabs=sample-data-storage%2Cmodel-aoai%2Cconnect-data-storage) and a sample guidelines file is enough.

If you want to skip this setup, either comment our the "Legal" agent or use the `telco_callcenter_sequence` notebook instead.


In [ ]:
%load_ext autoreload
%autoreload 2

In [61]:
# Add the parent directory to sys.path
import sys, os
sys.path.append(os.path.abspath(os.path.join('../vanilla_aiagents')))

from vanilla_aiagents.agent import Agent
from vanilla_aiagents.user import User
from vanilla_aiagents.workflow import Workflow
from vanilla_aiagents.team import Team
from vanilla_aiagents.llm import AzureOpenAILLM
from dotenv import load_dotenv

load_dotenv(override=True)
import os

In [ ]:
llm = AzureOpenAILLM({
    "azure_deployment": os.getenv("AZURE_OPENAI_MODEL"),
    "azure_endpoint": os.getenv("AZURE_OPENAI_ENDPOINT"),
    "api_key": os.getenv("AZURE_OPENAI_KEY"),
    "api_version": os.getenv("AZURE_OPENAI_API_VERSION"),
})

# Set logging to debug for Agent, User and Workflow
import logging

# Set logging to debug for Agent, User, and Workflow
logging.basicConfig(level=logging.INFO)
logging.getLogger("vanilla_aiagents.agent").setLevel(logging.DEBUG)
logging.getLogger("vanilla_aiagents.user").setLevel(logging.DEBUG)
logging.getLogger("vanilla_aiagents.workflow").setLevel(logging.DEBUG)

In [63]:
user = User(mode="unattended", description = "An interface to a human participant. Call this agent to get inputs from the user and proceed with the workflow.")

In [ ]:
greeter = Agent(description="A greeter agent, to say hello and welcome to the user", id="greeter", llm=llm, system_message="""
You are a call center operator that responds to customer inquiries. 
    
    Your task are:
    - Greet the Customer at first. Be sure to ask how you can help.
    - Check if the Customer has any additional questions. If not, close the conversation.
    - Close the conversation after the Customer's request has been resolved. Thank the Customer for their time and wish them a good day and write TERMINATE to end the conversation. DO write TERMINATE in the response.
    
    IMPORTANT NOTES:
    - Make sure to act politely and professionally.    
    - Make sure to write TERMINATE to end the conversation.    
    - NEVER pretend to act on behalf of the company. NEVER provide false information.
 """)

In [ ]:
sales = Agent(description="A sales agent that can answer sales questions", id="sales",llm=llm, system_message="""
You are a sales person that responds to customer inquiries.
    
    You have access to pricing and product details in the PRODUCTS sections below. Please note field starting with "_" are not to be shared with the Customer.
    
    Your tasks are:
    - provide the Customer with the information they need. Try to be specific and provide the customer only options that fit their needs.
    
    IMPORTANT NOTES:
    - DO act politely and professionally
    - NEVER provide false information
    
    ### PRODUCTS
    - Mobile Internet
        - Description: Mobile WiFi for you to take anywhere, supports up to 10 devices.
        - Price: €10/month
        - Details: 10GB data included, €1/GB after that.
        - _SKU: INET_MOBILE
    - All-in-One Bundle
        - Description: Mobile internet and home internet in one package.
        - Price: €45/month
        - Details: 10GB mobile data, €1/GB after that. Home internet included.
        - _SKU: INET_BUNDLE
    - Home Internet
        - Description: High-speed internet for your home.
        - Price: €30/month
        - Details: Unlimited data at 1Gbps.
        - _SKU: INET_HOME""")

In [ ]:
from typing_extensions import Annotated
from azure.search.documents import SearchClient
from azure.search.documents.models import VectorizableTextQuery
from azure.identity import DefaultAzureCredential
from azure.core.credentials import AzureKeyCredential

key = os.getenv("AZURE_SEARCH_ADMIN_KEY")
credential = DefaultAzureCredential() if key is None or key == "" else AzureKeyCredential(key)
search_client = SearchClient(
    endpoint=os.getenv("AZURE_SEARCH_ENDPOINT"),
    index_name=os.getenv("AZURE_SEARCH_INDEX"),
    credential=credential
)


legal = Agent(description="A legal agent that can answer legal or contractual questions", id="legal", llm=llm, system_message="""
You are a legal advisor that responds to customer inquiries.
Your tasks are:
- provide the Customer with the information they need
- ALWAYS look up information in the legal database before answering
- be specific and provide the customer only options that fit their needs
- ALWAYS include the source of your information in your response

IMPORTANT NOTES:
Make sure to act politely and professionally.   
NEVER provide false information
Make sure to read the instructions above again before answering
""")

@legal.register_tool(description="Query the legal database")
def query_legal_database(query: Annotated[str, "The query to search for in the legal database"]) -> Annotated[str, "Relevant documentation from the legal database"]:
    vector_query = VectorizableTextQuery(text=query, k_nearest_neighbors=1, fields="text_vector", exhaustive=True)
    search_results = search_client.search(
        search_text=query,  
        vector_queries= [vector_query],
        select=["title", "chunk_id", "chunk"],
        top=5
    )
    
    # Chunk id has format {parent_id}_pages_{page_number}
    
    sources_formatted = "\n".join([f'# Source "{document["title"]}" - Page {document["chunk_id"].split("_")[-1]}\n{document["chunk"]}' for document in search_results])
    
    return sources_formatted
    

In [ ]:
from typing_extensions import Annotated


technical = Agent(description="A technical support agent that can answer technical questions", id="legal", llm=llm, system_message="""You are a technical support agent that responds to customer inquiries.
    
    Your task are:
    - Assess the technical issue the customer is facing.
    - Verify if there any known issues with the service the customer is using.
    - Check remote telemetry data to identify potential issues with customer's device. Be sure to ask customer code first.
    - Provide the customer with possible solutions to the issue. See the list of common issues below.
    - When the service status is OK, reply the customer and suggest to restart the device.
    - When the service status is DEGRADED, apologize to the customer and kindly ask them to wait for the issue to be resolved.
    - Open an internal ticket if the issue cannot be resolved immediately.
    
    Make sure to act politely and professionally.    
    
    ### Common issues and solutions:

    - Home Internet:
        - Issue: No internet connection.
        - Solutions: 
            - Check the router's power supply and cables.
            - Restart the router.
            - Check the internet connection status LED on the router.
    - Mobile Internet:
        - Issue: Slow internet connection or no connection.
        - Solutions:
            - Check the signal strength on the device.
            - Restart the device.
            - Check the data usage on the device.
            - Suggest the customer to purchase additional data when the limit is reached.
    - All-in-One Bundle:
        USE a combination of the solutions for Home Internet and Mobile Internet.
    
    """)

@technical.register_tool(description="Get the service status")
def get_service_status(
    service_sku: Annotated[str, "The SKU of the service to check status for"]
    ) -> Annotated[str, "Status of the specified service"]:
        
    return "Service degraded"

@technical.register_tool(description="Get the service status")
def check_customer_telemetry(
    service_sku: Annotated[str, "The SKU of the service to check status for, values can be INET_MOBILE, INET_BUNDLE, INET_HOME"],
    customerCode: Annotated[str, "The customer code to check telemetry for"]
    ) -> Annotated[str, "Telemetry summary for the specified customer"]:
            
    return "No issues detected"

In [ ]:
team = Team(id="team", description="", members=[user, greeter, sales, technical, legal], llm=llm, stop_callback=lambda msgs: len(msgs) > 6)
workflow = Workflow(askable=team)

In [ ]:
workflow.restart()
workflow.run("Hi")
workflow.conversation.messages[-1]

In [ ]:
# workflow.run("I would like to know more about the All-in-One Bundle", restart=False)
workflow.run("I have problems with my home internet")
# workflow.run("I want to pay invoices every two months, does terms and condition allow that?")
workflow.conversation.messages[-1]

In [ ]:
workflow.run("No connection")
workflow.conversation.messages[-1]

In [ ]:
workflow.run("My customer code is 1234")
workflow.conversation.messages[-1]

In [ ]:
workflow.conversation.messages